In [1]:
# Set up the mount of the google drive.
!pip install librosa

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
import wave
import sys
import os
import librosa
import librosa.display
import xgboost as xgb
from  sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import sklearn.naive_bayes as nb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import kd_tree
import seaborn as sn
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.kd_tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packag

In [4]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint


from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

Using TensorFlow backend.


In [0]:
import pandas as pd
demographic = pd.read_csv('/content/drive/My Drive/CSCE 666/Project/respiratory-sound-database/select_patient_complete_COPD_info.csv', header=None)
demographic_data = demographic.values
patient_disease_labels = demographic_data[:, -1]
#demographic_data = demographic_data[0:, 1:-1] # Remove the id column (the first column)
select_patient_ids = list(demographic_data[:, 0])

In [6]:
print(len(demographic_data))
print(type(demographic_data))
print(demographic_data[0, :])
print(demographic_data.shape[1]  )
print(demographic_data[[1,3,4],:])
print(select_patient_ids)

117
<class 'numpy.ndarray'>
[101.      3.      0.     19.386   0.   ]
5
[[102.      0.75    0.     18.39    0.   ]
 [104.     70.      0.     28.47    1.   ]
 [105.      7.      0.     17.558   0.   ]]
[101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161.0, 162.0, 163.0, 165.0, 166.0, 167.0, 168.0, 169.0, 170.0, 171.0, 172.0, 174.0, 175.0, 176.0, 177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 184.0, 185.0, 186.0, 187.0, 188.0, 189.0, 191.0, 192.0, 193.0, 194.0, 195.0, 196.0, 198.0, 199.0, 200.0, 201.0, 202.0, 203.0, 204.0, 205.0, 206.0, 207.0, 208.0, 209.0, 210.0, 211.0, 212.0, 213.0, 214.0, 215.0, 216.0, 218.0, 219.0, 220.0, 221.0,

In [0]:
"""
# 8 disease classification
def updatelabel(labels): 
    l=[]
    for label in labels:
      if label == 0:
        a = np.array([1, 0, 0, 0, 0, 0, 0, 0])
      elif label == 1:
        a = np.array([0, 1, 0, 0, 0, 0, 0, 0])
      elif label == 2:
        a = np.array([0, 0, 1, 0, 0, 0, 0, 0])
      elif label == 3:
        a = np.array([0, 0, 0, 1, 0, 0, 0, 0])  
      elif label == 4:
        a = np.array([0, 0, 0, 0, 1, 0, 0, 0])  
      elif label == 5:
        a = np.array([0, 0, 0, 0, 0, 1, 0, 0])
      elif label == 6:
        a = np.array([0, 0, 0, 0, 0, 0, 1, 0])
      else:
        a = np.array([0, 0, 0, 0, 0, 0, 0, 1])
      l.append(a)
    return l
  """
# COPD vs. non-COPD
def updatelabel(labels):
    l=[]
    for label in labels:
      if label == 0:
        a = np.array([1, 0])
      if label == 1:
        a = np.array([0, 1]) 
      l.append(a)
    return l

In [8]:
print(patient_disease_labels)

new_patient_disease_labels = updatelabel(patient_disease_labels)
print(len(new_patient_disease_labels))
print(new_patient_disease_labels)

[0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0.
 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0.
 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0.]
117
[array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([1

In [9]:
print(demographic_data[0])
print(demographic_data[0, 1:-1])
print(len(demographic_data))
print(demographic_data[0, 0])
print(demographic_data[0, -1])

[101.      3.      0.     19.386   0.   ]
[ 3.     0.    19.386]
117
101.0
0.0


In [0]:
# Extract features
featset = demographic_data[0:, 1:-1] # Remove the id column (the first column)


In [11]:
#print(featset[0])
print(len(featset))
print(len(featset[0]))

117
3


In [12]:
new_patient_disease_labels = np.asarray(new_patient_disease_labels)
a = np.zeros(new_patient_disease_labels.shape[0])
for i in range(new_patient_disease_labels.shape[0]):
    for j in range(new_patient_disease_labels.shape[1]):
        if new_patient_disease_labels[i][j]==1:
            a[i]=j

# COnvert to binary for the binary classification problem
bin_label =np.zeros(new_patient_disease_labels.shape[0])
for index,l in enumerate(a):
    if l == 0:
        bin_label[index] = l
    else:
        bin_label[index] = 1

unique, counts = np.unique(bin_label, return_counts=True)

print(np.asarray((unique, counts)).T)


# One-hot encode labels
le = LabelEncoder()
i_labels = le.fit_transform(bin_label)
oh_labels = to_categorical(i_labels)


[[ 0. 55.]
 [ 1. 62.]]


In [0]:
scaler=StandardScaler()
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(featset, i_labels,test_size=0.3, random_state=42,stratify=i_labels)
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [14]:
# SVM
Cs = [2**(-2),2**(-1), 1,2**(1),2**(2),2**(3),2**(4),2**(5),2**(6),2**(7),2**(8),2**(9),2**(10)]
gamma = [2**(-7),2**(-6),2**(-5),2**(-4),2**(-3),2**(-2),2**(-1),2**(0),2**(1),2**(2),2**(3)]


param_grid = {'C': Cs,  
                'gamma': gamma, 
                'kernel': ['rbf'],
                'decision_function_shape':['ov'],
                'class_weight': ['balanced']}  
grid1 = GridSearchCV(SVC(), param_grid,cv=3,n_jobs=-1, verbose = 3) 

# fitting the model for grid search 
grid1.fit(x_train, y_train)

# print best parameter after tuning 
print(grid1.best_params_) 
# print how our model looks after hyper-parameter tuning 
print(grid1.best_estimator_)

grid_predictions = grid1.predict(x_test) 

# print classification report 
print(classification_report(y_test, grid_predictions))
print(accuracy_score(y_test,grid_predictions))

print(sklearn.metrics.confusion_matrix(y_test,grid_predictions))

Fitting 3 folds for each of 143 candidates, totalling 429 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 102 tasks      | elapsed:    1.0s


{'C': 0.25, 'class_weight': 'balanced', 'decision_function_shape': 'ov', 'gamma': 0.03125, 'kernel': 'rbf'}
SVC(C=0.25, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ov', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       1.00      0.76      0.87        17
           1       0.83      1.00      0.90        19

    accuracy                           0.89        36
   macro avg       0.91      0.88      0.89        36
weighted avg       0.91      0.89      0.89        36

0.8888888888888888
[[13  4]
 [ 0 19]]


[Parallel(n_jobs=-1)]: Done 429 out of 429 | elapsed:    1.5s finished


In [15]:
# XgBoost
import xgboost as xgb

print('Training XGB Classifier from new features:')
xgb_model = xgb.XGBClassifier(max_depth=3,num_class=2, n_estimators=15, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='multi:softmax', eta=0.3, silent=0, subsample=0.8).fit(x_train, y_train)

xgb_pred = xgb_model.predict(x_test)
# print classification report 
print(classification_report(y_test, xgb_pred))
print(accuracy_score(y_test,xgb_pred))

Training XGB Classifier from new features:
              precision    recall  f1-score   support

           0       0.87      0.76      0.81        17
           1       0.81      0.89      0.85        19

    accuracy                           0.83        36
   macro avg       0.84      0.83      0.83        36
weighted avg       0.84      0.83      0.83        36

0.8333333333333334


In [16]:
# ANN

# One hot the labels
le = LabelEncoder()

y_test = to_categorical(y_test, 2)
y_train = to_categorical(y_train, 2)

# Custom model for classification
model = Sequential()

model.add(Dense(3, activation='relu', input_shape=(3,),kernel_initializer='random_normal'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid')) 

# Compile the model
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam') 

model.summary()

#train model
num_epochs = 1000
num_batch_size = 128



callbacks = [
    ModelCheckpoint(
        filepath = '/content/drive/My Drive/CSCE 666/Project/Eric\'s Notebooks/saved_models/Demographic_ANN_Binary_{epoch:02d}.h5',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs,
          validation_data=(x_test, y_test), callbacks=callbacks, verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
dropout (Dropout)            (None, 3)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 8         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
1/1 [==============================] - ETA: 0s - loss: 0.7048 - accuracy: 0.4691
Epoch 00001: val_accuracy improved from -inf to 0.22222, saving model to /content/drive/My Drive/CSCE 666/Project/Eric's Notebooks/saved_models/Demographic_ANN_Binary_01.h5
1/1 [==============================] - 0s 427ms/step - loss: 0.7048 - accuracy: 0.4691 - val_loss: 0.7052 - val_accuracy: 0

In [17]:
model_predictions = model.predict(x_test) 
model_labels = model_predictions
#print(model_predictions[5])
for i in range(model_predictions.shape[0]):
  if model_predictions[i][0] > 0.5:
    model_labels[i][0] = 1
    model_labels[i][1] = 0
  else:
    model_labels[i][0] = 0
    model_labels[i][1] = 1

#print(model_labels[5])
# print classification report 
print(classification_report(y_test, model_predictions))
print(accuracy_score(y_test,model_predictions))

              precision    recall  f1-score   support

           0       0.87      0.76      0.81        17
           1       0.81      0.89      0.85        19

   micro avg       0.83      0.83      0.83        36
   macro avg       0.84      0.83      0.83        36
weighted avg       0.84      0.83      0.83        36
 samples avg       0.83      0.83      0.83        36

0.8333333333333334
